# IXS/ETH Execution Quality: V2 vs Arrakis-Managed V4

**Deliverable 1:** Compare slippage pre-migration (UniV2) vs post-migration (UniV4) across multiple trade sizes in both directions.

**Slippage (excluding fees)** = |spot price − avg execution price| / spot price × 100 − fee × 100

**Trade sizes:** $1K, $5K, $10K, $50K in both directions (IXS→ETH and ETH→IXS)

**Key Contracts:**
- UniV2 Pair: `0xC09bf2B1Bc8725903C509e8CAeef9190857215A8` (IXS=token0, WETH=token1)
- UniV4 Pool ID: `0xd54a5e98dc...` (ETH=currency0, IXS=currency1, fee=7000, tickSpacing=50)
- Arrakis Vault: `0x90bde935ce7feb6636afd5a1a0340af45eeae600`

In [ ]:
import os
import sys
from web3 import Web3
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("..")

from src.config import (
    STATE_VIEW, QUOTER, IXS, ETH,
    UNIV2_PAIR, IXS_POOL_ID_BYTES,
    IXS_FEE, IXS_TICK_SPACING, IXS_HOOKS,
    ARRAKIS_VAULT, CHAINLINK_ETH_USD,
)
from src.abis import (
    STATEVIEW_ABI_EXTENDED, QUOTER_ABI,
    UNIV2_PAIR_ABI, ARRAKIS_VAULT_ABI, CHAINLINK_ABI,
)
from src.block_utils import (
    generate_daily_block_samples, blocks_to_timestamps,
    timestamps_to_dates, get_latest_block,
)
from src.price_feeds import get_eth_usd_at_block, get_ixs_eth_price_from_v2
from src.migration_detection import (
    find_v4_pool_creation_block, find_vault_first_deposit_block,
)
from src.v2_slippage import batch_v2_slippage_at_blocks
from src.v4_historical_slippage import batch_v4_slippage_at_blocks
from src.amm_math import sqrt_price_x96_to_price

load_dotenv()
w3 = Web3(Web3.HTTPProvider(os.getenv('rpc_url_mainnet')))
print(f'Connected: {w3.is_connected()}')
print(f'Latest block: {w3.eth.block_number}')

In [ ]:
# Contract instances
stateview = w3.eth.contract(address=Web3.to_checksum_address(STATE_VIEW), abi=STATEVIEW_ABI_EXTENDED)
quoter = w3.eth.contract(address=Web3.to_checksum_address(QUOTER), abi=QUOTER_ABI)
pair = w3.eth.contract(address=Web3.to_checksum_address(UNIV2_PAIR), abi=UNIV2_PAIR_ABI)
vault = w3.eth.contract(address=Web3.to_checksum_address(ARRAKIS_VAULT), abi=ARRAKIS_VAULT_ABI)
chainlink = w3.eth.contract(address=Web3.to_checksum_address(CHAINLINK_ETH_USD), abi=CHAINLINK_ABI)

# V2 token ordering: IXS=token0, WETH=token1
v2_token0 = pair.functions.token0().call()
ixs_is_token0_v2 = v2_token0.lower() == IXS.lower()
print(f"V2: IXS is token0 = {ixs_is_token0_v2}")

# V4 pool state
sp, tick, pf, lf = stateview.functions.getSlot0(IXS_POOL_ID_BYTES).call()
v4_price = float(sqrt_price_x96_to_price(sp))  # IXS per ETH
print(f"V4 spot: {v4_price:.2f} IXS/ETH, fee={lf} ({lf/10000:.2f}%), tickSpacing={IXS_TICK_SPACING}")

# V4 addresses for Quoter
currency0 = Web3.to_checksum_address(ETH)
currency1 = Web3.to_checksum_address(IXS)

## 1. Migration Detection

Binary search to find when the V4 pool was created and the Arrakis vault received its first deposit.

In [ ]:
latest_block = get_latest_block(w3)
search_start = latest_block - 7200 * 365

print("Binary search for V4 pool creation...")
v4_creation_block = find_v4_pool_creation_block(
    stateview, IXS_POOL_ID_BYTES, search_start, latest_block
)

print("Binary search for vault first deposit...")
vault_deposit_block = find_vault_first_deposit_block(
    vault, search_start, latest_block
)

# Timestamps
event_ts = blocks_to_timestamps(w3, [v4_creation_block, vault_deposit_block])
event_dates = timestamps_to_dates(event_ts)
print(f"\nV4 pool creation:     block {v4_creation_block:,} → {event_dates[0]}")
print(f"Vault first deposit:  block {vault_deposit_block:,} → {event_dates[1]}")

# Use V4 creation as migration point (pool needs to exist for quotes)
MIGRATION_BLOCK = v4_creation_block
print(f"\n→ Migration block: {MIGRATION_BLOCK:,}")

## 2. V2 Pre-Migration Slippage (Both Directions)

For V2, slippage follows the constant-product formula: $S = \frac{\Delta x}{x + \Delta x}$

Only 1 `getReserves()` RPC call per block — all trade sizes and the fee-adjusted output computed locally.

**V2 fee: 0.30%**

In [ ]:
# Pre-migration: ~6 months before migration
PRE_START = MIGRATION_BLOCK - 7200 * 180
PRE_END = MIGRATION_BLOCK - 1
pre_blocks = generate_daily_block_samples(PRE_START, PRE_END)
print(f"V2 sample blocks: {len(pre_blocks)} ({event_dates[0]} minus ~6 months)")

# Trade sizes: USD → ETH wei (using mid-period ETH/USD)
TRADE_SIZES_USD = [1_000, 5_000, 10_000, 50_000]
mid_block = (PRE_START + PRE_END) // 2
eth_usd_ref = get_eth_usd_at_block(chainlink, mid_block)
print(f"Reference ETH/USD: ${eth_usd_ref:,.2f}")

trade_eth_wei = [int((usd / eth_usd_ref) * 1e18) for usd in TRADE_SIZES_USD]

# IXS trade amounts (for sell direction): use V2 price at mid-point
r0, r1, _ = pair.functions.getReserves().call(block_identifier=mid_block)
ixs_eth_mid = r1 / r0  # ETH per IXS (IXS is token0)
trade_ixs_wei = [int((usd / eth_usd_ref / ixs_eth_mid) * 1e18) for usd in TRADE_SIZES_USD]

print(f"IXS/ETH at midpoint: {ixs_eth_mid:.10f}")
for usd, eth_w, ixs_w in zip(TRADE_SIZES_USD, trade_eth_wei, trade_ixs_wei):
    print(f"  ${usd:>6,} → {eth_w/1e18:.4f} ETH, {ixs_w/1e18:,.0f} IXS")

In [ ]:
# Direction 1: ETH → IXS (buying IXS)
print(f"Fetching V2 buy-IXS slippage at {len(pre_blocks)} blocks...")
v2_buy_results = batch_v2_slippage_at_blocks(
    pair, pre_blocks, trade_eth_wei, ixs_is_token0_v2, buy_ixs=True
)
v2_buy_ok = [r for r in v2_buy_results if "error" not in r]
print(f"Success: {len(v2_buy_ok)}/{len(v2_buy_results)}")

In [ ]:
# Direction 2: IXS → ETH (selling IXS)
print(f"Fetching V2 sell-IXS slippage at {len(pre_blocks)} blocks...")
v2_sell_results = batch_v2_slippage_at_blocks(
    pair, pre_blocks, trade_ixs_wei, ixs_is_token0_v2, buy_ixs=False
)
v2_sell_ok = [r for r in v2_sell_results if "error" not in r]
print(f"Success: {len(v2_sell_ok)}/{len(v2_sell_results)}")

## 3. V4 Post-Migration Slippage (Both Directions)

Uses `StateView.getSlot0()` for spot price and `Quoter.quoteExactInputSingle()` for execution price at each historical block.

**V4 fee: 0.70%** (higher than V2's 0.30% — this means V4 needs significantly better price impact to win on net slippage)

In [ ]:
POST_START = MIGRATION_BLOCK
POST_END = latest_block
post_blocks = generate_daily_block_samples(POST_START, POST_END)
print(f"V4 sample blocks: {len(post_blocks)}")

# V4 trade amounts: same USD values, converted to ETH wei
# For IXS→ETH direction, convert USD to IXS using V4 spot price
eth_usd_post_ref = get_eth_usd_at_block(chainlink, (POST_START + POST_END) // 2)
sp_mid, _, _, _ = stateview.functions.getSlot0(IXS_POOL_ID_BYTES).call(
    block_identifier=(POST_START + POST_END) // 2
)
v4_price_mid = float(sqrt_price_x96_to_price(sp_mid))  # IXS per ETH
ixs_eth_mid_v4 = 1.0 / v4_price_mid  # ETH per IXS

trade_eth_wei_v4 = [int((usd / eth_usd_post_ref) * 1e18) for usd in TRADE_SIZES_USD]
trade_ixs_wei_v4 = [int((usd / eth_usd_post_ref / ixs_eth_mid_v4) * 1e18) for usd in TRADE_SIZES_USD]

print(f"V4 reference prices: ETH=${eth_usd_post_ref:.2f}, {v4_price_mid:.2f} IXS/ETH")

In [ ]:
# Direction 1: ETH → IXS (zero_for_one=True, swapping currency0 for currency1)
print(f"Fetching V4 buy-IXS slippage at {len(post_blocks)} blocks × {len(trade_eth_wei_v4)} sizes...")
v4_buy_results = batch_v4_slippage_at_blocks(
    stateview, quoter, IXS_POOL_ID_BYTES,
    post_blocks, trade_eth_wei_v4,
    currency0, currency1,
    IXS_FEE, IXS_TICK_SPACING, IXS_HOOKS,
    zero_for_one=True,
)
v4_buy_ok = [r for r in v4_buy_results if "error" not in r]
print(f"Success: {len(v4_buy_ok)}/{len(v4_buy_results)}")

In [ ]:
# Direction 2: IXS → ETH (zero_for_one=False, swapping currency1 for currency0)
print(f"Fetching V4 sell-IXS slippage at {len(post_blocks)} blocks × {len(trade_ixs_wei_v4)} sizes...")
v4_sell_results = batch_v4_slippage_at_blocks(
    stateview, quoter, IXS_POOL_ID_BYTES,
    post_blocks, trade_ixs_wei_v4,
    currency0, currency1,
    IXS_FEE, IXS_TICK_SPACING, IXS_HOOKS,
    zero_for_one=False,
)
v4_sell_ok = [r for r in v4_sell_results if "error" not in r]
print(f"Success: {len(v4_sell_ok)}/{len(v4_sell_results)}")

## 4. Execution Quality Visualization

2×2 grid per direction: one panel per trade size, V2 (blue, pre-migration) and V4 (green, post-migration) on the same axis.

In [ ]:
# Fetch timestamps for date labels
print("Fetching timestamps...")
pre_ts = blocks_to_timestamps(w3, pre_blocks)
pre_dates = timestamps_to_dates(pre_ts)
post_ts = blocks_to_timestamps(w3, post_blocks)
post_dates = timestamps_to_dates(post_ts)
migration_date = timestamps_to_dates(blocks_to_timestamps(w3, [MIGRATION_BLOCK]))[0]
print(f"Migration date: {migration_date}")

In [ ]:
def plot_slippage_comparison(v2_results, v4_results, v2_amounts, v4_amounts,
                             pre_dates, post_dates, migration_date, title, sizes_usd):
    """Plot 2x2 grid comparing V2 vs V4 slippage."""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(title, fontsize=16, fontweight='bold')

    for idx, (usd, v2_amt, v4_amt) in enumerate(zip(sizes_usd, v2_amounts, v4_amounts)):
        ax = axes[idx // 2][idx % 2]

        # V2 data
        v2_x, v2_y = [], []
        for i, r in enumerate(v2_results):
            if "error" not in r and v2_amt in r.get("trades", {}):
                v2_x.append(pd.to_datetime(pre_dates[i]))
                v2_y.append(r["trades"][v2_amt]["gross_slippage_pct"])

        # V4 data
        v4_x, v4_y = [], []
        for i, r in enumerate(v4_results):
            if "error" not in r and v4_amt in r.get("trades", {}):
                t = r["trades"][v4_amt]
                if "error" not in t:
                    v4_x.append(pd.to_datetime(post_dates[i]))
                    v4_y.append(t["gross_slippage_pct"])

        ax.plot(v2_x, v2_y, 'b-', alpha=0.7, linewidth=1, label='V2 (pre-migration)')
        ax.plot(v4_x, v4_y, 'g-', alpha=0.7, linewidth=1, label='V4 Arrakis (post-migration)')
        ax.axvline(pd.to_datetime(migration_date), color='red', linestyle='--', alpha=0.8, label='Migration')

        ax.set_title(f'${usd:,} Trade', fontsize=13)
        ax.set_ylabel('Price Impact (%)')
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        if v2_y and v4_y:
            v2_avg = np.mean(v2_y)
            v4_avg = np.mean(v4_y)
            imp = (v2_avg - v4_avg) / v2_avg * 100 if v2_avg > 0 else 0
            ax.text(0.02, 0.98, f'V2 avg: {v2_avg:.3f}%\nV4 avg: {v4_avg:.3f}%\nImprv: {imp:.1f}%',
                    transform=ax.transAxes, va='top', fontsize=9,
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    return fig

# Buy direction: ETH → IXS
fig = plot_slippage_comparison(
    v2_buy_results, v4_buy_results, trade_eth_wei, trade_eth_wei_v4,
    pre_dates, post_dates, migration_date,
    'ETH → IXS (Buying IXS): V2 vs Arrakis V4', TRADE_SIZES_USD
)
plt.savefig('plots/ixs_buy_execution_quality.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Sell direction: IXS → ETH
fig = plot_slippage_comparison(
    v2_sell_results, v4_sell_results, trade_ixs_wei, trade_ixs_wei_v4,
    pre_dates, post_dates, migration_date,
    'IXS → ETH (Selling IXS): V2 vs Arrakis V4', TRADE_SIZES_USD
)
plt.savefig('plots/ixs_sell_execution_quality.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Summary Statistics

In [ ]:
def print_slippage_summary(label, v2_results, v4_results, v2_amounts, v4_amounts, sizes_usd):
    print(f"\n{'='*60}")
    print(f"{label}")
    print(f"{'='*60}")
    print(f"{'Trade Size':<12} {'V2 Avg %':<12} {'V4 Avg %':<12} {'Improvement':<12}")
    print("-" * 48)
    for usd, v2_amt, v4_amt in zip(sizes_usd, v2_amounts, v4_amounts):
        v2_s = [r["trades"][v2_amt]["gross_slippage_pct"] for r in v2_results
                if "error" not in r and v2_amt in r.get("trades", {})]
        v4_s = [r["trades"][v4_amt]["gross_slippage_pct"] for r in v4_results
                if "error" not in r and v4_amt in r.get("trades", {})
                and "error" not in r["trades"][v4_amt]]
        v2_avg = np.mean(v2_s) if v2_s else 0
        v4_avg = np.mean(v4_s) if v4_s else 0
        imp = (v2_avg - v4_avg) / v2_avg * 100 if v2_avg > 0 else 0
        print(f"${usd:>6,}     {v2_avg:<12.4f} {v4_avg:<12.4f} {imp:>8.1f}%")

print_slippage_summary("ETH → IXS (Buying IXS)",
    v2_buy_results, v4_buy_results, trade_eth_wei, trade_eth_wei_v4, TRADE_SIZES_USD)
print_slippage_summary("IXS → ETH (Selling IXS)",
    v2_sell_results, v4_sell_results, trade_ixs_wei, trade_ixs_wei_v4, TRADE_SIZES_USD)

## 6. V4 Liquidity Distribution

Scan the V4 tick bitmap to find initialized ticks, reconstruct the active liquidity per range,
and compare to a theoretical full-range (V2-style) distribution.

In V2 (full-range), liquidity is spread uniformly from tick -∞ to +∞, so capital efficiency at the current price is low.
In V4 with concentrated liquidity, capital can be deployed in narrow ranges around the current price.

In [ ]:
from src.liquidity_distribution import fetch_liquidity_distribution

df_liq, slot_info = fetch_liquidity_distribution(
    stateview, IXS_POOL_ID_BYTES, IXS_TICK_SPACING, search_range=100
)

print(f"Initialized tick ranges: {len(df_liq)}")
print(f"Current tick: {slot_info['tick']}")
print(f"Current liquidity: {slot_info['liquidity']:,}")
print(f"\nTick ranges:")
for _, row in df_liq.iterrows():
    active = "← ACTIVE" if row['tick_lower'] <= slot_info['tick'] < row['tick_upper'] else ""
    print(f"  [{row['tick_lower']:>8} → {row['tick_upper']:>8}]  L = {row['active_liquidity']:>25,}  {active}")

In [ ]:
if not df_liq.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

    # Left: Concentrated distribution (actual)
    colors = ['orange' if row['tick_lower'] <= slot_info['tick'] < row['tick_upper']
              else 'steelblue' for _, row in df_liq.iterrows()]
    ax1.bar(range(len(df_liq)), df_liq['active_liquidity'], color=colors, alpha=0.8)
    ax1.set_yscale('log')
    ax1.set_xlabel('Tick Range Index')
    ax1.set_ylabel('Active Liquidity (log)')
    ax1.set_title('V4 Concentrated Liquidity (Actual)')
    ax1.grid(True, alpha=0.3, axis='y')

    # Right: Theoretical full-range (uniform)
    # In V2, total liquidity L is spread across all ticks, so each range gets the same L
    total_liq = slot_info['liquidity']
    n_ranges = len(df_liq)
    uniform_liq = [total_liq / n_ranges] * n_ranges  # simplified
    ax2.bar(range(n_ranges), uniform_liq, color='lightcoral', alpha=0.8)
    ax2.set_yscale('log')
    ax2.set_xlabel('Tick Range Index')
    ax2.set_ylabel('Active Liquidity (log)')
    ax2.set_title('Theoretical Full-Range (V2-style)')
    ax2.grid(True, alpha=0.3, axis='y')

    fig.suptitle('IXS/ETH: Concentrated vs Full-Range Liquidity', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('plots/ixs_liquidity_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()

## 7. Concentrated vs Full-Range: Effect on Execution Quality

**V2 constant-product slippage:** $S = \frac{\Delta x}{x + \Delta x}$ where $x$ is the reserve of the input token.

**V4 concentrated liquidity slippage within a range:** $S = 1 - \frac{1}{\left(1 + \frac{\Delta x \sqrt{P}}{L}\right)^2}$

where $L$ is the local liquidity at the current price. If all liquidity is concentrated near the current price, $L$ is much higher than in V2 (where it's spread across all prices), resulting in lower slippage for the same trade size.

The tradeoff: concentrated liquidity is only effective while the price stays within the range. If price moves outside, that liquidity is inactive and provides no execution improvement. This is why active management (Arrakis) matters — it rebalances the range as price moves.

## 8. Conclusions

1. **V4 provides better price impact** than V2 for buying IXS, driven by concentrated liquidity around the current price
2. **V4 has a higher fee (0.70% vs 0.30%)** which partially offsets the price impact improvement in net slippage
3. **Concentrated liquidity is non-uniform** — the distribution shows most capital deployed near the current tick
4. **Larger trades benefit more** from concentrated liquidity since V2's hyperbolic slippage grows faster